In [1]:
# Importing modules
import torch as to
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch.utils.data as to_data
from torch.utils.tensorboard import SummaryWriter as sumwriter
import os as os

In [2]:
# Specify hardware for ML training (GPU default)
device = "cuda" if to.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# Quickly generate list of strings for frequency numbers and ratios
def freq_name(no_freq, include_freq=True, include_ratio=True):
    """
    Creates an ordered list of string from inputted parameters:

    no_freq = (int) number of desired frequencies
    include_freq = (bool) include the individual frequencies or not (default True)
    include_ratio = (bool) include the non-trivial ratios between frequencies or not (default True)
    """
    names = []
    if include_freq:
        for i in range(no_freq):
            names.append('f'+str(i+1))
    if include_ratio:
        for i in range(no_freq):
            for j in range(i):
                names.append('f'+str(i+1)+'/f'+str(j+1))
    return names

In [4]:
# Create Pytorch dataset class for data batching during training
class FFNN_data(to_data.Dataset):
    def __init__(self, scaled_dataframe, X_names, Y_names):
        self.len = len(scaled_dataframe)
        self.numfreqs = len(X_names)
        self.X = to.from_numpy(scaled_dataframe[X_names].to_numpy().astype('float32')).to(device)
        self.Y = to.from_numpy(scaled_dataframe[Y_names].to_numpy().astype('float32')).to(device)

    def __len__(self):
        return self.len
  
    def __getitem__(self, idx):
        shuffl_inds = to.randperm(self.numfreqs, device=device)
        X_idx = self.X[idx,:][shuffl_inds]
        # mode_nums = to.flatten(to.tril(to.ones((self.numfreqs,self.numfreqs), device=device))[shuffl_inds,:])
        # X_idx = to.cat((mode_nums, X_idx))
        X_idx = to.cat((shuffl_inds, X_idx))
        Y_idx = self.Y[idx,:]
        return X_idx, Y_idx

In [5]:
# Creates function that returns desired activation function
def activation(activ_name):
    if activ_name=='relu':
        return to.nn.ReLU()
    elif activ_name=='lrelu':
        return to.nn.LeakyReLU()
    elif activ_name=='prelu':
        return to.nn.PReLU()
    elif activ_name=='relu6':
        return to.nn.ReLU6()
    elif activ_name=='sigmoid':
        return to.nn.Sigmoid()
    elif activ_name=='tanh':
        return to.nn.Tanh()
    elif activ_name=='silu':
        return to.nn.SiLU()
    elif activ_name=='selu':
        return to.nn.SELU()
    elif activ_name=='celu':
        return to.nn.CELU()
    elif activ_name=='gelu':
        return to.nn.GELU()
    else:
        return to.nn.ReLU()

In [6]:
class FF_Network(to.nn.Module):
    def __init__(self, num_X, num_Y, h_nodes, hactiv_type):
        super(FF_Network, self).__init__()

        self.feedfoward = []
        self.feedfoward.append(to.nn.Linear(num_X, h_nodes[0]))
        self.feedfoward.append(activation(hactiv_type))

        for i in range(len(h_nodes)-1):
            self.feedfoward.append(to.nn.Linear(h_nodes[i], h_nodes[i+1]))
            self.feedfoward.append(activation(hactiv_type))

        self.feedfoward.append(to.nn.Linear(h_nodes[-1], num_Y))

        self.feedfoward = to.nn.Sequential(*self.feedfoward).to(device)
        for i in self.feedfoward[::2]:
            to.nn.init.kaiming_uniform_(i.weight)
            to.nn.init.zeros_(i.bias)

    def forward(self, X):
        Y = self.feedfoward(X)
        return Y

In [7]:
def train_epoch(
    network,
    train_dataloader,
    loss_function, optimizer,
    tb_writer, epoch_ind
    ):

    loss_list = []
    MAPE_list = []

    for i, data in enumerate(train_dataloader):
        X, Y = data

        if epoch_ind==0 and i==0:
            tb_writer.add_graph(network, X, verbose=False)

        optimizer.zero_grad()
        predictY = network(X)

        loss = loss_function(predictY, Y)
        loss_list.append(loss.item())

        loss.backward()
        optimizer.step()

        MAPE = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPE_list.append(MAPE.item())

    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_MAPE = to.mean(to.tensor(MAPE_list, device=device)).item()

    return mean_loss, mean_MAPE

def valid_epoch(
    network,
    valid_dataloader,
    loss_function
    ):

    loss_list = []
    MAPE_list = []

    for i, data in enumerate(valid_dataloader):
        X, Y = data
        predictY = network(X)

        loss = loss_function(predictY, Y)
        loss_list.append(loss.item())
 
        MAPE = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPE_list.append(MAPE.item())
    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_MAPE = to.mean(to.tensor(MAPE_list, device=device)).item()
    return mean_loss, mean_MAPE

In [8]:
def train_AEI(
    network,
    train_dataloader, valid_dataloader,
    loss_function, optimizer_type,
    epochs, learn_rate
    ):

    if optimizer_type=='adam':
        optimizer = to.optim.Adam(network.parameters(), lr=learn_rate)
    else:
        optimizer = to.optim.SGD(network.parameters(), lr=learn_rate)
    
    tb_writer = sumwriter('Current_ML_Results/Tensorboard')
    
    for i in range(epochs):
        network.train(True)
        mloss, mMAPE = train_epoch(network, train_dataloader, loss_function, optimizer, tb_writer, i)

        network.eval()
        with to.no_grad():
            vmloss, vmMAPE = valid_epoch(network, valid_dataloader, loss_function)
        

        print('-'*50)
        print('Epoch {} / {}'.format(i+1,epochs))
        print('-'*15)
        print('Average Train Loss : {}'.format(mloss))
        print('Average Validation Loss : {}'.format(vmloss))

        tb_writer.add_scalars("Batch Mean Loss",
                            {
                                'Train' : mloss,
                                'Validation' : vmloss
                            }, i+1)

        tb_writer.add_scalars("Batch MAPE",
                            {
                                'Train' : mMAPE,
                                'Validation' : vmMAPE
                            }, i+1)

    tb_writer.flush()
    tb_writer.close()

In [9]:
# Prepare Data
data = pd.read_csv('Data Workspace/FM_TV_Data.csv')
num_freq = 5

features = freq_name(num_freq,1,0)
labels = ['psi', 'nu', 'a', 'b']

train_split = int(0.8*len(data))
valid_split = len(data)- train_split

scaled_data = data[features+labels].copy()
scaled_data[freq_name(num_freq,1,0)] = np.log(scaled_data[freq_name(num_freq,1,0)])
scaled_data['psi'] = np.log(scaled_data['psi'])
# scaled_data['t'] = scaled_data['t']*100
# scaled_data['E'] = scaled_data['E']/1e11
# scaled_data['rho'] = scaled_data['rho']/10000

scaled_data = FFNN_data(scaled_data, features, labels)
train_set, valid_set = to_data.random_split(scaled_data, [train_split, valid_split])

In [15]:
# Train Model
# Parameters
num_X = len(features)+len(features)
num_Y = len(labels)
h_nodes = [20,30,40,50,60,50,40,30,20,10,6]
hactiv = 'prelu'

batch_size_train = 2000
batch_size_valid = 2000

epochs = 100
learn_rate = 1e-3

# Optim Selections
loss_function = to.nn.SmoothL1Loss()
optimizer_type = 'adam'

# Data loaders
train_loader = to.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=True)
valid_loader = to.utils.data.DataLoader(valid_set, batch_size=batch_size_valid, shuffle=True)

# Model
model = FF_Network(num_X, num_Y, h_nodes, hactiv)

In [16]:
# Train
train_AEI(
    model,
    train_loader, valid_loader,
    loss_function, optimizer_type,
    epochs, learn_rate)

to.save(model.state_dict(), 'Current_ML_Results/model.state')

--------------------------------------------------
Epoch 1 / 100
---------------
Average Train Loss : 0.8244194984436035
Average Validation Loss : 0.5230622887611389
--------------------------------------------------
Epoch 2 / 100
---------------
Average Train Loss : 0.47519874572753906
Average Validation Loss : 0.4286484718322754
--------------------------------------------------
Epoch 3 / 100
---------------
Average Train Loss : 0.34353911876678467
Average Validation Loss : 0.22955931723117828
--------------------------------------------------
Epoch 4 / 100
---------------
Average Train Loss : 0.14830079674720764
Average Validation Loss : 0.10247163474559784
--------------------------------------------------
Epoch 5 / 100
---------------
Average Train Loss : 0.09865142405033112
Average Validation Loss : 0.09563860297203064
--------------------------------------------------
Epoch 6 / 100
---------------
Average Train Loss : 0.09334126114845276
Average Validation Loss : 0.0948089361190

In [17]:
# model.load_state_dict(to.load('Current_ML_Results/model.state'))

trainvalid_data = pd.read_csv('Data Workspace/FM_TV_Data.csv')
scaled_trainvalid_data = trainvalid_data[features+labels].copy()

scaled_trainvalid_data['psi'] = np.log(scaled_trainvalid_data['psi'])
# scaled_trainvalid_data['E'] = scaled_trainvalid_data['E']/1e11
# scaled_trainvalid_data['rho'] = scaled_trainvalid_data['rho']/10000
# scaled_trainvalid_data['t'] = scaled_trainvalid_data['t']*100
scaled_trainvalid_data[freq_name(num_freq,1,0)] = np.log(scaled_trainvalid_data[freq_name(num_freq,1,0)])

scaled_trainvalid_data = FFNN_data(scaled_trainvalid_data, features, labels)
test_loader = to.utils.data.DataLoader(scaled_trainvalid_data, batch_size=len(scaled_trainvalid_data), shuffle=False)

model.eval()
with to.no_grad():
    for i, data in enumerate(test_loader):
        X, Y = data
        predictY = model.feedfoward(X)
            
        Y[:,0] = to.exp(Y[:,0])
        predictY[:,0] = to.exp(predictY[:,0])
        abs_perc_error = to.abs((Y- predictY)/Y)*100
        MAPE_per_dim = to.mean(abs_perc_error, 0)

        np.savetxt('Current_ML_Results/MAPE_trainvalid.txt', MAPE_per_dim.cpu().numpy())

        print('Absolute Percentage Errors: ')
        print('-'*20)
        print('f1: {:0.2f} %'.format(MAPE_per_dim[0].item()))
        print('f2: {:0.2f} %'.format(MAPE_per_dim[1].item()))
        print('f3: {:0.2f} %'.format(MAPE_per_dim[2].item()))
        print('f4: {:0.2f} %'.format(MAPE_per_dim[3].item()))
        # print('f5: {:0.2f} %'.format(MAPE_per_dim[4].item()))
        # print('f6: {:0.2f} %'.format(MAPE_per_dim[5].item()))
        # print('f7: {:0.2f} %'.format(MAPE_per_dim[6].item()))
        # print('f8: {:0.2f} %'.format(MAPE_per_dim[7].item()))
        # print('f9: {:0.2f} %'.format(MAPE_per_dim[8].item()))
        # print('f10: {:0.2f} %'.format(MAPE_per_dim[9].item()))

Absolute Percentage Errors: 
--------------------
f1: 72.90 %
f2: 16.93 %
f3: 49.19 %
f4: 48.67 %


In [18]:
# model.load_state_dict(to.load('Current_ML_Results/model.state'))

test_data = pd.read_csv('Data Workspace/FM_Te_Data.csv')
scaled_test_data = test_data[features+labels].copy()

scaled_test_data['psi'] = np.log(scaled_test_data['psi'])
scaled_test_data[freq_name(num_freq,1,0)] = np.log(scaled_test_data[freq_name(num_freq,1,0)])

scaled_test_data = FFNN_data(scaled_test_data, features, labels)
test_loader = to.utils.data.DataLoader(scaled_test_data, batch_size=len(scaled_test_data), shuffle=False)

model.eval()
with to.no_grad():
    for i, data in enumerate(test_loader):
        X, Y = data
        predictY = model.feedfoward(X)
            
        Y[:,0] = to.exp(Y[:,0])
        predictY[:,0] = to.exp(predictY[:,0])
        abs_perc_error = to.abs((Y- predictY)/Y)*100
        MAPE_per_dim = to.mean(abs_perc_error, 0)

        np.savetxt('Current_ML_Results/MAPE_test.txt', MAPE_per_dim.cpu().numpy())

        print('Absolute Percentage Errors: ')
        print('-'*20)
        print('f1: {:0.2f} %'.format(MAPE_per_dim[0].item()))
        print('f2: {:0.2f} %'.format(MAPE_per_dim[1].item()))
        print('f3: {:0.2f} %'.format(MAPE_per_dim[2].item()))
        print('f4: {:0.2f} %'.format(MAPE_per_dim[3].item()))
        # print('f5: {:0.2f} %'.format(MAPE_per_dim[4].item()))
        # print('f6: {:0.2f} %'.format(MAPE_per_dim[5].item()))
        # print('f7: {:0.2f} %'.format(MAPE_per_dim[6].item()))
        # print('f8: {:0.2f} %'.format(MAPE_per_dim[7].item()))
        # print('f9: {:0.2f} %'.format(MAPE_per_dim[8].item()))
        # print('f10: {:0.2f} %'.format(MAPE_per_dim[9].item()))

Absolute Percentage Errors: 
--------------------
f1: 73.86 %
f2: 17.06 %
f3: 48.86 %
f4: 49.30 %
